In [1]:
import gc
import glob
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import utils
import utils_fe
from gbm_pipeline import GBMPipeline
from tqdm import tqdm

/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: insp

/home/w/anaconda3/envs/idp3exp/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/w/anaconda3/envs/idp3exp/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/lightgbm/__init__.py:27: ResourceWarning: unclosed file <_io.TextIOWrapper name='/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/lightgbm/VERSION.txt' mode='r' encoding='UTF-8'>
  __version__ = open(os.path.join(dir_path, 'VERSION.txt')).read().strip()


In [2]:
src_oof = 'oof/'
src_preds = '/home/w/Projects/Toxic/scripts/predictions/'
src_features = '../data/features/'

run_kfold = True
split_for_validation = True
add_features = True

run_kfold = 1

In [3]:
if run_kfold:
    prefix = 'KFold'
else:
    prefix = 'Bag'


train = pd.read_pickle("../data/train_basic_clean.pkl")
test = pd.read_pickle("../data/test_basic_clean.pkl")
target_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

oof_tr, oof_te = utils.load_predictions(src_oof, contains='Check')
oof_tr, oof_te = np.asarray(oof_tr), np.asarray(oof_te)
print(oof_tr.shape, oof_te.shape)

X_train_oof = pd.DataFrame()
X_test_oof = pd.DataFrame()

for i in range(oof_tr.shape[0]):
    oof_tr_part = oof_tr[i][:, :, 0]
    oof_te_part = oof_te[i].mean(axis=-1)[:, :]
    X_train_oof = pd.concat([X_train_oof, pd.DataFrame(oof_tr_part)], axis=1)
    X_test_oof = pd.concat([X_test_oof, pd.DataFrame(oof_te_part)], axis=1)


f_cols = []

for i in range((X_train_oof.shape[1] // 6)):
    f_cols.extend(['{}_preds_model{}'.format(target_columns[x], i) for x in range(len(target_columns))])

X_train_oof.columns = f_cols
X_test_oof.columns = f_cols


if add_features:
    data_tokenized = pd.read_pickle(src_features + 'data_TokenizedSentences160.pkl')
    data_badwords300 = pd.read_pickle(src_features + 'data_Binary300Badwords.pkl')
    data_textStatistics = pd.read_pickle(src_features + 'data_TextStatistics.pkl')
    data_transformations = pd.read_pickle(src_features + 'data_20dim_SVDLSA.pkl')

    data_tokenized.reset_index(drop=True, inplace=True)
    
    X = pd.concat([data_tokenized, data_badwords300, data_textStatistics, data_transformations], axis=1)
    X['badwordsCount'] = data_badwords300.sum(axis=1)

    X_train = X.iloc[:train.shape[0], :]
    X_test = X.iloc[train.shape[0]:, :]
    y_train = train[target_columns].values
    
    X_test.reset_index(drop=True, inplace=True)
    features = np.setdiff1d(X_train.columns, target_columns)

    X_train = pd.concat([X_train[features], X_train_oof], axis=1)
    X_test = pd.concat([X_test[features], X_test_oof], axis=1)
    
    del X, test
    del data_tokenized, data_badwords300
    del data_textStatistics, data_transformations
    gc.collect()

else:
    X_train = X_train_oof
    y_train = train[target_columns].values
    X_test = X_test_oof
    

print(X_train.shape, X_test.shape)


Train files:

['oof/train/GRUattentionBidirectional256_1stSet_KFold1_Glove_BasicClean_200dim_300k_256len_random0_5fold_BS512_Nadam_fromCheck_121.12740.pkl',
 'oof/train/GRUbasicBidirectional256_1stSet_KFold1_Glove_BasicClean_200dim_300k_256len_random0_5fold_BS512_Nadam_fromCheck_110.98027.pkl',
 'oof/train/GRUconcatBidirectional256_1stSet_KFold1_Glove_BasicClean_200dim_300k_256len_random0_5fold_BS512_Nadam_fromCheck_94.57426.pkl',
 'oof/train/LSTMattentionBidirectional256_1stSet_KFold1_Glove_BasicClean_200dim_300k_256len_random0_5fold_BS512_Adam_fromCheck_116.43669.pkl',
 'oof/train/LSTMconcatBidirectional256_1stSet_KFold1_Glove_BasicClean_200dim_300k_256len_random0_5fold_BS512_Adam_fromCheck_113.36922.pkl']

Test files:

['oof/test/GRUattentionBidirectional256_1stSet_KFold1_Glove_BasicClean_200dim_300k_256len_random0_5fold_BS512_Nadam_fromCheck_121.12740.pkl',
 'oof/test/GRUbasicBidirectional256_1stSet_KFold1_Glove_BasicClean_200dim_300k_256len_random0_5fold_BS512_Nadam_fromCheck_110

In [4]:
features

array([' damn', ' dyke', ' fuck ', ' shit ', ' ss',
       'LSA_dim0_comment_text', 'LSA_dim10_comment_text',
       'LSA_dim11_comment_text', 'LSA_dim12_comment_text',
       'LSA_dim13_comment_text', 'LSA_dim14_comment_text',
       'LSA_dim15_comment_text', 'LSA_dim16_comment_text',
       'LSA_dim17_comment_text', 'LSA_dim18_comment_text',
       'LSA_dim19_comment_text', 'LSA_dim1_comment_text',
       'LSA_dim2_comment_text', 'LSA_dim3_comment_text',
       'LSA_dim4_comment_text', 'LSA_dim5_comment_text',
       'LSA_dim6_comment_text', 'LSA_dim7_comment_text',
       'LSA_dim8_comment_text', 'LSA_dim9_comment_text',
       'SVD_dim0_comment_text', 'SVD_dim10_comment_text',
       'SVD_dim11_comment_text', 'SVD_dim12_comment_text',
       'SVD_dim13_comment_text', 'SVD_dim14_comment_text',
       'SVD_dim15_comment_text', 'SVD_dim16_comment_text',
       'SVD_dim17_comment_text', 'SVD_dim18_comment_text',
       'SVD_dim19_comment_text', 'SVD_dim1_comment_text',
       'SVD_dim2

In [5]:
xgb_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'eta': 0.05,
    'max_depth': 10,
    'min_child_weight': 20,
    'subsample': 0.8,
    'lambda': 0,
    'tree_method': 'hist',
    'nthread': 10,
    'silent': True,
}

lgb_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.05,
    'num_leaves': 255,
    'max_depth': 10,
    'min_child_weight': 20,
    'subsample': 0.8,
    'reg_lambda': 0,
    'nthread': 10,
}


if split_for_validation:
    
    train_params = {
        'boost_round': 10000,
        'stopping_rounds': 50,
        'verbose_eval': 50,
    }
    
else:
    
    train_params = {
        'boost_round': 161,
        'stopping_rounds': 50,
        'verbose_eval': False,
    }

    
pipeline_params = {
    'use_lgb': True,
    'predict_test': True,
    'seed': 1337,
    'shuffle': True,
    'verbose': True,
    'run_save_name': 'LGB_{}_StackerNoFeats'.format(prefix),
    'save_model': False,
    'save_history': False,
    'save_statistics': True,
    'output_statistics': True,
    'output_importance': True,
}


XGB_pipeline = GBMPipeline(
    use_lgb=pipeline_params['use_lgb'],
    predict_test=pipeline_params['predict_test'],
    seed=pipeline_params['seed'],
    shuffle=pipeline_params['shuffle'],
    verbose=pipeline_params['verbose'],
    run_save_name=pipeline_params['run_save_name'],
    save_model=pipeline_params['save_model'],
    save_history=pipeline_params['save_history'],
    save_statistics=pipeline_params['save_statistics'],
    output_statistics=pipeline_params['output_statistics'],
    output_importance=pipeline_params['output_importance'],
)


In [6]:
if pipeline_params['use_lgb']:
    gbm_params = lgb_params
else:
    gbm_params = xgb_params


if pipeline_params['predict_test']:
    val_preds, test_preds, gbm = XGB_pipeline.fold_run(X_train, train[target_columns],
                                           X_test=X_test,
                                           model_params=gbm_params,
                                           train_params=train_params,
                                           output_submission=True,
                                                      additional_features=features)
else:
    val_preds, gbm = XGB_pipeline.fold_run(X_train[features], y_train=X_train[targets],
                               model_params=gbm_params,
                               train_params=train_params)


utils.save_parameter_dict(
    'checkpoints/{0}/{0}_gbm_parameters.txt'.format(pipeline_params['run_save_name']), gbm_params)
utils.save_parameter_dict('checkpoints/{0}/{0}_train_parameters.txt'.format(
    pipeline_params['run_save_name']), train_params)
utils.save_parameter_dict('checkpoints/{0}/{0}_pipeline_parameters.txt'.format(
    pipeline_params['run_save_name']), pipeline_params)

Using LightGBM
OOF train predictions shape: (159571, 6)
X_train shape: (159571, 586)
OOF test predictions shape: (153164, 6, 5)
X_test shape: (153164, 586)
Running KFold run with 5 folds
Start training with parameters: {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 'learning_rate': 0.05, 'num_leaves': 255, 'max_depth': 10, 'min_child_weight': 20, 'subsample': 0.8, 'reg_lambda': 0, 'nthread': 10} 
 

Training model for column: toxic
Training until validation scores don't improve for 50 rounds.
[50]	valid_0's auc: 0.982341
[100]	valid_0's auc: 0.982448
Early stopping, best iteration is:
[80]	valid_0's auc: 0.982517
Minimum validation split loss for current fold/bag: 0.98251711604641 

Seconds it took to train the model: 2.0076451301574707 

Best iterations: [80] 

Visualize model feature importance.
Predicting on test data.
Training model for column: severe_toxic
Training until validation scores don't improve for 50 rounds.
[50]	valid_0's auc: 0.991189

Training model for column: insult
Training until validation scores don't improve for 50 rounds.
[50]	valid_0's auc: 0.98665
[100]	valid_0's auc: 0.986847
Early stopping, best iteration is:
[85]	valid_0's auc: 0.986981
Minimum validation split loss for current fold/bag: 0.9869805491120507 

Seconds it took to train the model: 39.65369272232056 

Best iterations: [80, 56, 145, 130, 71, 33, 108, 141, 197, 148, 18, 154, 138, 15, 159, 53, 85] 

Visualize model feature importance.
Predicting on test data.
Training model for column: identity_hate
Training until validation scores don't improve for 50 rounds.
[50]	valid_0's auc: 0.985291
[100]	valid_0's auc: 0.98628
[150]	valid_0's auc: 0.986767
Early stopping, best iteration is:
[116]	valid_0's auc: 0.987037
Minimum validation split loss for current fold/bag: 0.9870369114554057 

Seconds it took to train the model: 41.94359993934631 

Best iterations: [80, 56, 145, 130, 71, 33, 108, 141, 197, 148, 18, 154, 138, 15, 159, 53, 85, 116] 

Visualiz